In [2]:
%%time
import sys
sys.path.insert(0, '/Users/shuyafeng/PycharmProjects/xiaozaizi/codes/')

import os
import json
import warnings
import pprint


%reload_ext autoreload
%autoreload
%autoreload 2
%config Completer.use_jedi = False

CPU times: user 15.2 ms, sys: 13 ms, total: 28.3 ms
Wall time: 27.6 ms


In [9]:
import numpy as np
import scipy.io
from numpy.random import choice, laplace
from scipy.special import lambertw
from scipy.stats import laplace
from codes.kse_test import kse_test
from codes.dbscan import dbscan
from codes.forest import forest
import pandas as pd
import random


In [78]:
dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/4.mat')['dataset4']
sizeofD = 100
dataset2 = dataset4[:, :3]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]

Output = forest(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [82]:
EVAL = np.zeros((len(eps), 7))
#EVAL1 = np.zeros((4, 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]
        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        
        
        #noiseanom[noiseanom<0]=0
        noiseunif = forest(unif)
        noiseanom = forest(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [83]:
dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/4.mat')['dataset4']
sizeofD = 100
dataset2 = dataset4[:, :3]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]

Output = forest(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [84]:
EVAL = np.zeros((len(eps), 7))
#EVAL1 = np.zeros((4, 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=True, p=pp))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]
        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in noiseanom]
        
        
        #noiseanom[noiseanom<0]=0
        noiseunif = forest(unif)
        noiseanom = forest(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL1[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



## baseline

In [118]:
dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/4.mat')['dataset4']
#dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = forest(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [121]:
#EVAL = np.zeros((4, 7))
EVAL2 = np.zeros((len(eps), 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=False))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        

        noiseunif = forest(unif)
        noiseanom = forest(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL2[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [122]:
#dataset4 = scipy.io.loadmat('/Users/shuyafeng/PycharmProjects/xiaozaizi/server1.mat')['dataset4']
dataset4 = np.array(pd.read_csv('tr_server_data.csv'))
sizeofD = 100
dataset2 = dataset4[:, :1]
S, _ = np.histogram(dataset2, bins=sizeofD)
eps = [0.5,1,2,3,4,5,6,7,8,9,10]
B = S.reshape(-1, 1)
C = B[:sizeofD]
Output = forest(C)

pp = 1. / (Output ** 5)
pp = pp.reshape(-1)
pp /= pp.sum()
NN = 2 ** (15 * Output)
px = np.arange(1, sizeofD+1)

gamma = 0.5
rho = np.exp(lambertw(-gamma / (2 * np.exp(0.5)), k=-1) + 0.5)
rho = np.real(rho)
m = np.log(1 / rho) / (2 * (gamma - rho) ** 2)
k = m * (1 - gamma + rho + np.sqrt(np.log(1 / rho) / (2 * m)))
m = int(float(np.ceil(m)))
k = int(float(np.ceil(k)))


In [128]:
#EVAL = np.zeros((4, 7))
EVAL3 = np.zeros((len(eps), 7))

for rr in range(len(eps)):
    for j in range(1):
        C = B[:sizeofD].reshape(-1)
        
        GS = np.sort(np.random.choice(C, size=m, replace=False))
        Sensunif = GS[k-1]
        SS = np.sort(np.random.choice(px, size=m, replace=False))
        SS = NN[np.floor(SS).astype(int)]
        SS = np.sort(SS)
        Sensanom = SS[k-1]

        C= B[:sizeofD]
        #unif = laplace.rvs(loc=0, scale=Sensunif/(eps[rr]), size=sizeofD) + C
        unif=C
        unif=[i + laplace.rvs(loc=0, scale=Sensunif/(eps[rr]),size=1) for i in unif]

        #noiseanom = laplace.rvs(loc=0, scale=Sensanom/(eps[rr]), size=sizeofD) + NN
        noiseanom = NN
        noiseanom=[i + laplace.rvs(loc=0, scale=Sensanom/(eps[rr]),size=1) for i in noiseanom]
        

        noiseunif = forest(unif)
        noiseanom = forest(noiseanom)

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if Output[i] > 0.8:
                Index[i, 1] = 1

        ACTUAL = Index
        Origindex = np.where(ACTUAL[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseunif[i] > 0.8:
                Index[i, 1] = 1

        Uniformm = Index
        Pertindex = np.where(Uniformm[:, 1] == 1)[0]

        Index = np.zeros((sizeofD, 2))
        Index[:, 0] = np.arange(1, sizeofD+1)

        for i in range(sizeofD):
            if noiseanom[i] > 0.8:
                Index[i, 1] = 1

        Anomm = Index
        Pertindex1 = np.where(Anomm[:, 1] == 1)[0]
        IND1 = Anomm
        IND = Uniformm

        C = len(set(Origindex) | set(Pertindex))
        p = len(Origindex)
        n = sizeofD - p
        N = p + n
        tp = sum(ACTUAL[Origindex, 1] == IND[Origindex, 1])
        tn = N - C
        fp = n - tn
        fn = p - tp
        tp_rate = tp / p
        tn_rate = tn / n
        accuracy = (tp + tn) / N
        sensitivity = tp_rate
        specificity = tn_rate
        precision = tp / (tp + fp)
        recall = sensitivity
        f_measure = 2 * ((precision * recall) / (precision + recall))
        gmean = np.sqrt(tp_rate * tn)
        EVAL3[rr, :] += np.array([accuracy, tp, specificity, precision, recall, f_measure, gmean])



In [129]:
m=pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [114]:
m1=pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [130]:
m2=pd.DataFrame(EVAL2, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [131]:
m3=pd.DataFrame(EVAL3, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [132]:
to_save=[m['accuracy'],m['precision'],m['recall'],m['f_measure'],m1['accuracy'],m1['precision'],m1['recall'],m1['f_measure'],m2['accuracy'],m2['precision'],m2['recall'],m2['f_measure'],m3['accuracy'],m3['precision'],m3['recall'],m3['f_measure']]
to_save=pd.DataFrame(to_save).T
to_save.columns=['accuracy','precision','recall','f_measure','accuracy1','precision1','recall1','f_measure1','accuracy2','precision2','recall2','f_measure2','accuracy3','precision3','recall3','f_measure3']
to_save.to_csv('svm1.csv', index=False)


In [65]:
pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.49,39.0,0.196078,0.487500,0.795918,0.604651,2.821203
1,0.38,25.0,0.254902,0.396825,0.510204,0.446429,2.575394
2,0.45,26.0,0.372549,0.448276,0.530612,0.485981,3.175159
3,0.51,21.0,0.588235,0.500000,0.428571,0.461538,3.585686
4,0.56,12.0,0.862745,0.631579,0.244898,0.352941,3.282607
5,0.40,20.0,0.392157,0.392157,0.408163,0.400000,2.857143
6,0.45,17.0,0.549020,0.425000,0.346939,0.382022,3.116775
7,0.51,40.0,0.215686,0.500000,0.816327,0.620155,2.996597
8,0.47,26.0,0.411765,0.464286,0.530612,0.495238,3.338092
9,0.42,19.0,0.450980,0.404255,0.387755,0.395833,2.986364


In [66]:
pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

,accuracy,tp,specificity,precision,recall,f_measure,gmean
0,0.50,30.0,0.392157,0.491803,0.612245,0.545455,3.499271
1,0.56,26.0,0.588235,0.553191,0.530612,0.541667,3.989783
2,0.51,37.0,0.274510,0.500000,0.755102,0.601626,3.251373
3,0.48,28.0,0.392157,0.474576,0.571429,0.518519,3.380617
4,0.53,34.0,0.372549,0.515152,0.693878,0.591304,3.630933
5,0.64,27.0,0.725490,0.658537,0.551020,0.600000,4.515280
6,0.46,28.0,0.352941,0.459016,0.571429,0.509091,3.207135
7,0.46,22.0,0.470588,0.448980,0.448980,0.448980,3.282607
8,0.49,20.0,0.568627,0.476190,0.408163,0.439560,3.440456
9,0.56,32.0,0.470588,0.542373,0.653061,0.592593,3.958973


In [58]:
m=pd.DataFrame(EVAL, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [59]:
m1=pd.DataFrame(EVAL1, columns=["accuracy", "tp", "specificity", "precision", "recall", "f_measure", "gmean"])

In [60]:
to_save=[m['accuracy'],m['precision'],m['recall'],m['f_measure'],m1['accuracy'],m1['precision'],m1['recall'],m1['f_measure']]
to_save=pd.DataFrame(to_save).T
to_save.columns=['accuracy','precision','recall','f_measure','accuracy1','precision1','recall1','f_measure1',]
to_save.to_csv('svm.csv', index=False)